## Summary

##### We will investigate the performance of the CatBoost library on this problem. CatBoost is a state-of-the-art open-source gradient boosting on decision trees library. The following experiments will be performed:

##### 1) Test it's performance directly on the raw data
##### 2) Test it's performance once we have standardize features by removing the mean and scaling to unit variance (sklearn.preprocessing.StandardScaler)
##### 3) Test it's performance once we have transformed features by scaling each feature to a given range (0, 1) (sklearn.preprocessing.MinMaxScaler)
##### 4) Test performance using cross validation



### Visualising the data

#### What files are in the input folder:

In [1]:
# All imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import gc


In [2]:
import os 
print('No. of files: {}'.format(str(len(os.listdir("./input")))))
print(os.listdir("./input"))

No. of files: 3
['sample_submission.csv', 'test.csv', 'train.csv']


##### Train and Test files

In [85]:
#Loading Train and Test Data

df_train = pd.read_csv("./input/train.csv")
df_test = pd.read_csv("./input/test.csv")

print("{} observations and {} features in train set.".format(df_train.shape[0], df_train.shape[1]))
print("{} observations and {} features in test set.".format(df_test.shape[0], df_test.shape[1]))

200000 observations and 202 features in train set.
200000 observations and 201 features in test set.


In [12]:
df_train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [13]:
df_test.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


Let's now look at the target score 

In [14]:
df_train['target'].value_counts()

0    179902
1     20098
Name: target, dtype: int64

#### Predicting on RAW data

In [86]:
from sklearn.model_selection import train_test_split

#x = x_transformed_train
#y = df_train['target']
df_train_target = df_train['target']
df_train_modified = df_train.iloc[:,2:]
df_test_modified = df_test.iloc[:,1:]

x_train, x_test, y_train, y_test = train_test_split(df_train_modified, df_train_target, random_state = 13, test_size = 0.3)

pd.DataFrame(data=x_train).head(3)

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
61092,11.0156,-1.7036,13.6113,10.7284,11.3013,3.5456,4.6526,11.0551,-2.4123,5.8285,...,-0.4699,3.7281,0.7810,1.2648,23.2156,-1.2186,13.0670,8.4942,21.0695,-22.0377
77702,4.5837,2.8545,8.2859,6.8230,14.4487,2.0574,5.2793,14.4021,-4.9655,8.4677,...,6.7582,1.7876,3.2980,4.5590,14.3334,-2.0163,2.5545,8.2228,14.0187,1.9558
19517,9.2443,-7.7795,9.6477,8.1215,11.6240,-9.6645,5.6095,21.3256,2.8393,7.1223,...,4.3627,11.8427,3.2404,14.2788,22.8260,-2.9378,4.2337,9.6881,18.1063,-23.0591


In [20]:
pd.DataFrame(data=y_train).head(3)

,target
61092,0
77702,0
19517,0


In [21]:
pd.DataFrame(data=x_test).head(3)

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
55460,10.2572,2.2939,9.1147,7.9457,9.8134,-17.0926,5.0725,14.9654,2.8733,6.1608,...,12.4596,4.9281,2.5307,5.2673,19.8761,-1.1277,-7.2447,9.8530,17.9966,-20.7829
191859,12.1454,2.6294,10.3763,8.2464,10.8403,-17.3967,6.6690,17.1594,2.4456,9.4530,...,9.1293,2.4748,1.5009,6.2691,13.4652,-1.4229,-8.6105,9.2023,17.3172,0.0352
46956,18.1069,1.3129,9.4579,3.3247,11.4447,-16.7164,6.0643,12.8521,2.2359,7.4008,...,0.2891,6.1889,1.2743,7.7563,22.3554,-1.7241,4.3412,8.4876,16.5669,-8.2945


In [22]:
pd.DataFrame(data=y_test).head(3)

,target
55460,0
191859,1
46956,0


In [23]:
# Helper function for making float values in interval [0,1] binary
def to_binary(x):
    return (0 if x < 0.5 else 1)

In [56]:
import catboost

# iter: 800, learning_rate=0.02, depth=7, no scaling  --> 0.8776
# iter: 2000, learning_rate=0.03, depth=7, no scaling  --> 0.8959 TRUE: 0.645

CAT = catboost.CatBoostClassifier(iterations=2000, learning_rate=0.03, depth=7)


In [57]:
CAT.fit(x_train, y_train, verbose=True)

0:	learn: 0.6667398	total: 176ms	remaining: 5m 51s
1:	learn: 0.6420957	total: 327ms	remaining: 5m 26s
2:	learn: 0.6194617	total: 497ms	remaining: 5m 30s
3:	learn: 0.5984346	total: 686ms	remaining: 5m 42s
4:	learn: 0.5786616	total: 852ms	remaining: 5m 39s
5:	learn: 0.5602028	total: 1.02s	remaining: 5m 39s
6:	learn: 0.5435402	total: 1.19s	remaining: 5m 37s
7:	learn: 0.5279247	total: 1.36s	remaining: 5m 39s
8:	learn: 0.5132721	total: 1.56s	remaining: 5m 45s
9:	learn: 0.4997661	total: 1.71s	remaining: 5m 40s
10:	learn: 0.4870565	total: 1.87s	remaining: 5m 38s
11:	learn: 0.4747521	total: 2.03s	remaining: 5m 36s
12:	learn: 0.4635358	total: 2.21s	remaining: 5m 37s
13:	learn: 0.4529396	total: 2.37s	remaining: 5m 35s
14:	learn: 0.4435218	total: 2.54s	remaining: 5m 36s
15:	learn: 0.4348249	total: 2.71s	remaining: 5m 35s
16:	learn: 0.4261556	total: 2.84s	remaining: 5m 31s
17:	learn: 0.4180833	total: 3s	remaining: 5m 30s
18:	learn: 0.4104215	total: 3.16s	remaining: 5m 29s
19:	learn: 0.4035053	tota

158:	learn: 0.2684802	total: 22.8s	remaining: 4m 24s
159:	learn: 0.2682568	total: 23s	remaining: 4m 24s
160:	learn: 0.2680368	total: 23.1s	remaining: 4m 24s
161:	learn: 0.2678017	total: 23.3s	remaining: 4m 24s
162:	learn: 0.2675668	total: 23.5s	remaining: 4m 24s
163:	learn: 0.2673275	total: 23.6s	remaining: 4m 24s
164:	learn: 0.2671038	total: 23.8s	remaining: 4m 24s
165:	learn: 0.2668784	total: 23.9s	remaining: 4m 23s
166:	learn: 0.2666527	total: 24s	remaining: 4m 23s
167:	learn: 0.2664336	total: 24.2s	remaining: 4m 23s
168:	learn: 0.2662304	total: 24.3s	remaining: 4m 23s
169:	learn: 0.2660273	total: 24.4s	remaining: 4m 23s
170:	learn: 0.2658144	total: 24.6s	remaining: 4m 22s
171:	learn: 0.2656020	total: 24.7s	remaining: 4m 23s
172:	learn: 0.2653865	total: 24.9s	remaining: 4m 22s
173:	learn: 0.2651769	total: 25s	remaining: 4m 22s
174:	learn: 0.2649791	total: 25.2s	remaining: 4m 22s
175:	learn: 0.2647753	total: 25.3s	remaining: 4m 22s
176:	learn: 0.2645627	total: 25.5s	remaining: 4m 22s

315:	learn: 0.2427071	total: 45.5s	remaining: 4m 2s
316:	learn: 0.2425773	total: 45.6s	remaining: 4m 2s
317:	learn: 0.2424612	total: 45.8s	remaining: 4m 2s
318:	learn: 0.2423395	total: 45.9s	remaining: 4m 2s
319:	learn: 0.2422252	total: 46.1s	remaining: 4m 1s
320:	learn: 0.2420981	total: 46.2s	remaining: 4m 1s
321:	learn: 0.2419728	total: 46.4s	remaining: 4m 1s
322:	learn: 0.2418468	total: 46.5s	remaining: 4m 1s
323:	learn: 0.2417111	total: 46.7s	remaining: 4m 1s
324:	learn: 0.2415889	total: 46.8s	remaining: 4m 1s
325:	learn: 0.2414647	total: 47s	remaining: 4m 1s
326:	learn: 0.2413503	total: 47.1s	remaining: 4m 1s
327:	learn: 0.2412196	total: 47.3s	remaining: 4m
328:	learn: 0.2410959	total: 47.4s	remaining: 4m
329:	learn: 0.2409819	total: 47.5s	remaining: 4m
330:	learn: 0.2408566	total: 47.7s	remaining: 4m
331:	learn: 0.2407482	total: 47.8s	remaining: 4m
332:	learn: 0.2406368	total: 48s	remaining: 4m
333:	learn: 0.2405196	total: 48.1s	remaining: 3m 59s
334:	learn: 0.2403934	total: 48.2

472:	learn: 0.2262970	total: 1m 8s	remaining: 3m 40s
473:	learn: 0.2262107	total: 1m 8s	remaining: 3m 40s
474:	learn: 0.2261289	total: 1m 8s	remaining: 3m 40s
475:	learn: 0.2260386	total: 1m 8s	remaining: 3m 40s
476:	learn: 0.2259562	total: 1m 8s	remaining: 3m 40s
477:	learn: 0.2258677	total: 1m 9s	remaining: 3m 40s
478:	learn: 0.2257770	total: 1m 9s	remaining: 3m 39s
479:	learn: 0.2256939	total: 1m 9s	remaining: 3m 39s
480:	learn: 0.2255925	total: 1m 9s	remaining: 3m 39s
481:	learn: 0.2255076	total: 1m 9s	remaining: 3m 39s
482:	learn: 0.2254247	total: 1m 9s	remaining: 3m 39s
483:	learn: 0.2253428	total: 1m 10s	remaining: 3m 39s
484:	learn: 0.2252650	total: 1m 10s	remaining: 3m 39s
485:	learn: 0.2251716	total: 1m 10s	remaining: 3m 39s
486:	learn: 0.2250812	total: 1m 10s	remaining: 3m 39s
487:	learn: 0.2249973	total: 1m 10s	remaining: 3m 38s
488:	learn: 0.2249210	total: 1m 10s	remaining: 3m 38s
489:	learn: 0.2248271	total: 1m 10s	remaining: 3m 38s
490:	learn: 0.2247354	total: 1m 11s	rem

625:	learn: 0.2143795	total: 1m 31s	remaining: 3m 19s
626:	learn: 0.2143023	total: 1m 31s	remaining: 3m 19s
627:	learn: 0.2142239	total: 1m 31s	remaining: 3m 19s
628:	learn: 0.2141539	total: 1m 31s	remaining: 3m 19s
629:	learn: 0.2140876	total: 1m 31s	remaining: 3m 19s
630:	learn: 0.2140239	total: 1m 31s	remaining: 3m 19s
631:	learn: 0.2139541	total: 1m 31s	remaining: 3m 18s
632:	learn: 0.2138888	total: 1m 32s	remaining: 3m 18s
633:	learn: 0.2138113	total: 1m 32s	remaining: 3m 18s
634:	learn: 0.2137371	total: 1m 32s	remaining: 3m 18s
635:	learn: 0.2136617	total: 1m 32s	remaining: 3m 18s
636:	learn: 0.2135913	total: 1m 32s	remaining: 3m 18s
637:	learn: 0.2135171	total: 1m 32s	remaining: 3m 18s
638:	learn: 0.2134441	total: 1m 32s	remaining: 3m 18s
639:	learn: 0.2133840	total: 1m 33s	remaining: 3m 17s
640:	learn: 0.2133183	total: 1m 33s	remaining: 3m 17s
641:	learn: 0.2132483	total: 1m 33s	remaining: 3m 17s
642:	learn: 0.2131737	total: 1m 33s	remaining: 3m 17s
643:	learn: 0.2131119	total:

...

1238:	learn: 0.1847629	total: 3m 2s	remaining: 1m 51s
1239:	learn: 0.1847346	total: 3m 2s	remaining: 1m 51s
1240:	learn: 0.1846996	total: 3m 2s	remaining: 1m 51s
1241:	learn: 0.1846685	total: 3m 2s	remaining: 1m 51s
1242:	learn: 0.1846320	total: 3m 2s	remaining: 1m 51s
1243:	learn: 0.1845908	total: 3m 2s	remaining: 1m 51s
1244:	learn: 0.1845626	total: 3m 3s	remaining: 1m 51s
1245:	learn: 0.1845194	total: 3m 3s	remaining: 1m 50s
1246:	learn: 0.1844804	total: 3m 3s	remaining: 1m 50s
1247:	learn: 0.1844404	total: 3m 3s	remaining: 1m 50s
1248:	learn: 0.1844049	total: 3m 3s	remaining: 1m 50s
1249:	learn: 0.1843747	total: 3m 3s	remaining: 1m 50s
1250:	learn: 0.1843389	total: 3m 3s	remaining: 1m 50s
1251:	learn: 0.1843001	total: 3m 4s	remaining: 1m 50s
1252:	learn: 0.1842653	total: 3m 4s	remaining: 1m 49s
1253:	learn: 0.1842298	total: 3m 4s	remaining: 1m 49s
1254:	learn: 0.1841949	total: 3m 4s	remaining: 1m 49s
1255:	learn: 0.1841529	total: 3m 4s	remaining: 1m 49s
1256:	learn: 0.1841164	total

1389:	learn: 0.1797014	total: 3m 24s	remaining: 1m 29s
1390:	learn: 0.1796737	total: 3m 24s	remaining: 1m 29s
1391:	learn: 0.1796419	total: 3m 24s	remaining: 1m 29s
1392:	learn: 0.1796114	total: 3m 25s	remaining: 1m 29s
1393:	learn: 0.1795791	total: 3m 25s	remaining: 1m 29s
1394:	learn: 0.1795442	total: 3m 25s	remaining: 1m 29s
1395:	learn: 0.1795095	total: 3m 25s	remaining: 1m 28s
1396:	learn: 0.1794898	total: 3m 25s	remaining: 1m 28s
1397:	learn: 0.1794600	total: 3m 25s	remaining: 1m 28s
1398:	learn: 0.1794233	total: 3m 25s	remaining: 1m 28s
1399:	learn: 0.1793924	total: 3m 26s	remaining: 1m 28s
1400:	learn: 0.1793569	total: 3m 26s	remaining: 1m 28s
1401:	learn: 0.1793312	total: 3m 26s	remaining: 1m 27s
1402:	learn: 0.1792981	total: 3m 26s	remaining: 1m 27s
1403:	learn: 0.1792575	total: 3m 26s	remaining: 1m 27s
1404:	learn: 0.1792315	total: 3m 26s	remaining: 1m 27s
1405:	learn: 0.1792020	total: 3m 26s	remaining: 1m 27s
1406:	learn: 0.1791726	total: 3m 27s	remaining: 1m 27s
1407:	lear

1540:	learn: 0.1750944	total: 3m 46s	remaining: 1m 7s
1541:	learn: 0.1750632	total: 3m 46s	remaining: 1m 7s
1542:	learn: 0.1750410	total: 3m 46s	remaining: 1m 7s
1543:	learn: 0.1750110	total: 3m 46s	remaining: 1m 7s
1544:	learn: 0.1749719	total: 3m 47s	remaining: 1m 6s
1545:	learn: 0.1749446	total: 3m 47s	remaining: 1m 6s
1546:	learn: 0.1749152	total: 3m 47s	remaining: 1m 6s
1547:	learn: 0.1748877	total: 3m 47s	remaining: 1m 6s
1548:	learn: 0.1748578	total: 3m 47s	remaining: 1m 6s
1549:	learn: 0.1748216	total: 3m 47s	remaining: 1m 6s
1550:	learn: 0.1747932	total: 3m 48s	remaining: 1m 6s
1551:	learn: 0.1747673	total: 3m 48s	remaining: 1m 5s
1552:	learn: 0.1747377	total: 3m 48s	remaining: 1m 5s
1553:	learn: 0.1747023	total: 3m 48s	remaining: 1m 5s
1554:	learn: 0.1746771	total: 3m 48s	remaining: 1m 5s
1555:	learn: 0.1746438	total: 3m 48s	remaining: 1m 5s
1556:	learn: 0.1746111	total: 3m 48s	remaining: 1m 5s
1557:	learn: 0.1745834	total: 3m 49s	remaining: 1m 4s
1558:	learn: 0.1745567	total

1695:	learn: 0.1706716	total: 4m 9s	remaining: 44.7s
1696:	learn: 0.1706388	total: 4m 9s	remaining: 44.5s
1697:	learn: 0.1706099	total: 4m 9s	remaining: 44.4s
1698:	learn: 0.1705895	total: 4m 9s	remaining: 44.2s
1699:	learn: 0.1705610	total: 4m 9s	remaining: 44.1s
1700:	learn: 0.1705281	total: 4m 9s	remaining: 43.9s
1701:	learn: 0.1704962	total: 4m 10s	remaining: 43.8s
1702:	learn: 0.1704746	total: 4m 10s	remaining: 43.6s
1703:	learn: 0.1704499	total: 4m 10s	remaining: 43.5s
1704:	learn: 0.1704231	total: 4m 10s	remaining: 43.3s
1705:	learn: 0.1703919	total: 4m 10s	remaining: 43.2s
1706:	learn: 0.1703600	total: 4m 10s	remaining: 43s
1707:	learn: 0.1703359	total: 4m 10s	remaining: 42.9s
1708:	learn: 0.1703024	total: 4m 11s	remaining: 42.7s
1709:	learn: 0.1702727	total: 4m 11s	remaining: 42.6s
1710:	learn: 0.1702464	total: 4m 11s	remaining: 42.5s
1711:	learn: 0.1702213	total: 4m 11s	remaining: 42.3s
1712:	learn: 0.1701957	total: 4m 11s	remaining: 42.2s
1713:	learn: 0.1701726	total: 4m 11s

1848:	learn: 0.1665991	total: 4m 31s	remaining: 22.1s
1849:	learn: 0.1665744	total: 4m 31s	remaining: 22s
1850:	learn: 0.1665499	total: 4m 31s	remaining: 21.9s
1851:	learn: 0.1665300	total: 4m 31s	remaining: 21.7s
1852:	learn: 0.1665040	total: 4m 31s	remaining: 21.6s
1853:	learn: 0.1664805	total: 4m 31s	remaining: 21.4s
1854:	learn: 0.1664625	total: 4m 32s	remaining: 21.3s
1855:	learn: 0.1664343	total: 4m 32s	remaining: 21.1s
1856:	learn: 0.1664182	total: 4m 32s	remaining: 21s
1857:	learn: 0.1663949	total: 4m 32s	remaining: 20.8s
1858:	learn: 0.1663750	total: 4m 32s	remaining: 20.7s
1859:	learn: 0.1663489	total: 4m 32s	remaining: 20.5s
1860:	learn: 0.1663226	total: 4m 32s	remaining: 20.4s
1861:	learn: 0.1662890	total: 4m 33s	remaining: 20.2s
1862:	learn: 0.1662689	total: 4m 33s	remaining: 20.1s
1863:	learn: 0.1662421	total: 4m 33s	remaining: 19.9s
1864:	learn: 0.1662213	total: 4m 33s	remaining: 19.8s
1865:	learn: 0.1661956	total: 4m 33s	remaining: 19.6s
1866:	learn: 0.1661634	total: 4m

In [58]:
y_pred = CAT.predict_proba(x_test) # for classifier
print(y_pred[:5])

[[0.98861992 0.01138008]
 [0.83970891 0.16029109]
 [0.98168692 0.01831308]
 [0.9043182  0.0956818 ]
 [0.61015241 0.38984759]]


In [59]:
from sklearn.metrics import roc_auc_score

roc_test = roc_auc_score(y_test, y_pred[:, 1:])
print(roc_test)

0.8959342464725196


In [61]:
prediction = CAT.predict_proba(df_test_modified)

prediction_CAT = [to_binary(x[0]) for x in prediction[:, 1:]]
print(sum(prediction_CAT))
submission = pd.read_csv('./input/sample_submission.csv')
submission['target'] = prediction_CAT
submission.to_csv('CAT_raw.csv', index=False)

5094


#### Standard Scaler

In [79]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(copy=True)
df_train_std = scaler.fit_transform(df_train.iloc[:,2:])
df_test_std = scaler.transform(df_test.iloc[:,1:])

In [80]:
x_train_std, x_test_std, y_train_std, y_test_std = train_test_split(df_train_std, df_train_target, random_state = 13, test_size = 0.3)


In [81]:
CAT_std = catboost.CatBoostClassifier(iterations=2000, learning_rate=0.03, depth=7)

In [82]:
CAT_std.fit(x_train_std, y_train_std, verbose=True)

0:	learn: 0.6667398	total: 180ms	remaining: 6m
1:	learn: 0.6420957	total: 334ms	remaining: 5m 33s
2:	learn: 0.6194617	total: 503ms	remaining: 5m 34s
3:	learn: 0.5984346	total: 695ms	remaining: 5m 46s
4:	learn: 0.5786616	total: 862ms	remaining: 5m 43s
5:	learn: 0.5602028	total: 1.03s	remaining: 5m 42s
6:	learn: 0.5435402	total: 1.2s	remaining: 5m 41s
7:	learn: 0.5279247	total: 1.37s	remaining: 5m 42s
8:	learn: 0.5132721	total: 1.57s	remaining: 5m 46s
9:	learn: 0.4997661	total: 1.71s	remaining: 5m 41s
10:	learn: 0.4870565	total: 1.88s	remaining: 5m 39s
11:	learn: 0.4747521	total: 2.04s	remaining: 5m 38s
12:	learn: 0.4635358	total: 2.25s	remaining: 5m 43s
13:	learn: 0.4529396	total: 2.42s	remaining: 5m 43s
14:	learn: 0.4435218	total: 2.61s	remaining: 5m 45s
15:	learn: 0.4348249	total: 2.78s	remaining: 5m 44s
16:	learn: 0.4261556	total: 2.91s	remaining: 5m 39s
17:	learn: 0.4180833	total: 3.08s	remaining: 5m 38s
18:	learn: 0.4104215	total: 3.23s	remaining: 5m 36s
19:	learn: 0.4035053	total:

159:	learn: 0.2682568	total: 23.3s	remaining: 4m 28s
160:	learn: 0.2680368	total: 23.5s	remaining: 4m 28s
161:	learn: 0.2678017	total: 23.6s	remaining: 4m 28s
162:	learn: 0.2675668	total: 23.8s	remaining: 4m 28s
163:	learn: 0.2673275	total: 23.9s	remaining: 4m 28s
164:	learn: 0.2671038	total: 24.1s	remaining: 4m 27s
165:	learn: 0.2668784	total: 24.2s	remaining: 4m 27s
166:	learn: 0.2666527	total: 24.3s	remaining: 4m 27s
167:	learn: 0.2664336	total: 24.5s	remaining: 4m 26s
168:	learn: 0.2662304	total: 24.6s	remaining: 4m 26s
169:	learn: 0.2660273	total: 24.8s	remaining: 4m 26s
170:	learn: 0.2658144	total: 24.9s	remaining: 4m 26s
171:	learn: 0.2656020	total: 25s	remaining: 4m 26s
172:	learn: 0.2653865	total: 25.2s	remaining: 4m 25s
173:	learn: 0.2651769	total: 25.3s	remaining: 4m 25s
174:	learn: 0.2649791	total: 25.5s	remaining: 4m 25s
175:	learn: 0.2647753	total: 25.6s	remaining: 4m 25s
176:	learn: 0.2645627	total: 25.7s	remaining: 4m 25s
177:	learn: 0.2643603	total: 25.9s	remaining: 4m

317:	learn: 0.2424612	total: 46s	remaining: 4m 3s
318:	learn: 0.2423395	total: 46.2s	remaining: 4m 3s
319:	learn: 0.2422252	total: 46.3s	remaining: 4m 3s
320:	learn: 0.2420981	total: 46.5s	remaining: 4m 3s
321:	learn: 0.2419728	total: 46.6s	remaining: 4m 2s
322:	learn: 0.2418468	total: 46.8s	remaining: 4m 2s
323:	learn: 0.2417111	total: 46.9s	remaining: 4m 2s
324:	learn: 0.2415889	total: 47.1s	remaining: 4m 2s
325:	learn: 0.2414647	total: 47.2s	remaining: 4m 2s
326:	learn: 0.2413503	total: 47.4s	remaining: 4m 2s
327:	learn: 0.2412196	total: 47.5s	remaining: 4m 2s
328:	learn: 0.2410959	total: 47.6s	remaining: 4m 1s
329:	learn: 0.2409819	total: 47.8s	remaining: 4m 1s
330:	learn: 0.2408566	total: 48s	remaining: 4m 1s
331:	learn: 0.2407482	total: 48.1s	remaining: 4m 1s
332:	learn: 0.2406368	total: 48.2s	remaining: 4m 1s
333:	learn: 0.2405196	total: 48.4s	remaining: 4m 1s
334:	learn: 0.2403934	total: 48.5s	remaining: 4m 1s
335:	learn: 0.2402767	total: 48.7s	remaining: 4m 1s
336:	learn: 0.24

475:	learn: 0.2260386	total: 1m 8s	remaining: 3m 40s
476:	learn: 0.2259562	total: 1m 9s	remaining: 3m 40s
477:	learn: 0.2258677	total: 1m 9s	remaining: 3m 40s
478:	learn: 0.2257770	total: 1m 9s	remaining: 3m 40s
479:	learn: 0.2256939	total: 1m 9s	remaining: 3m 40s
480:	learn: 0.2255925	total: 1m 9s	remaining: 3m 40s
481:	learn: 0.2255076	total: 1m 9s	remaining: 3m 39s
482:	learn: 0.2254247	total: 1m 9s	remaining: 3m 39s
483:	learn: 0.2253428	total: 1m 10s	remaining: 3m 39s
484:	learn: 0.2252650	total: 1m 10s	remaining: 3m 39s
485:	learn: 0.2251716	total: 1m 10s	remaining: 3m 39s
486:	learn: 0.2250812	total: 1m 10s	remaining: 3m 39s
487:	learn: 0.2249973	total: 1m 10s	remaining: 3m 39s
488:	learn: 0.2249210	total: 1m 10s	remaining: 3m 38s
489:	learn: 0.2248271	total: 1m 10s	remaining: 3m 38s
490:	learn: 0.2247354	total: 1m 11s	remaining: 3m 38s
491:	learn: 0.2246494	total: 1m 11s	remaining: 3m 38s
492:	learn: 0.2245564	total: 1m 11s	remaining: 3m 38s
493:	learn: 0.2244749	total: 1m 11s	

628:	learn: 0.2141539	total: 1m 31s	remaining: 3m 20s
629:	learn: 0.2140876	total: 1m 32s	remaining: 3m 20s
630:	learn: 0.2140239	total: 1m 32s	remaining: 3m 19s
631:	learn: 0.2139541	total: 1m 32s	remaining: 3m 19s
632:	learn: 0.2138888	total: 1m 32s	remaining: 3m 19s
633:	learn: 0.2138113	total: 1m 32s	remaining: 3m 19s
634:	learn: 0.2137371	total: 1m 32s	remaining: 3m 19s
635:	learn: 0.2136617	total: 1m 32s	remaining: 3m 19s
636:	learn: 0.2135913	total: 1m 33s	remaining: 3m 19s
637:	learn: 0.2135171	total: 1m 33s	remaining: 3m 19s
638:	learn: 0.2134441	total: 1m 33s	remaining: 3m 18s
639:	learn: 0.2133840	total: 1m 33s	remaining: 3m 18s
640:	learn: 0.2133183	total: 1m 33s	remaining: 3m 18s
641:	learn: 0.2132483	total: 1m 33s	remaining: 3m 18s
642:	learn: 0.2131737	total: 1m 34s	remaining: 3m 18s
643:	learn: 0.2131119	total: 1m 34s	remaining: 3m 18s
644:	learn: 0.2130453	total: 1m 34s	remaining: 3m 18s
645:	learn: 0.2129762	total: 1m 34s	remaining: 3m 18s
646:	learn: 0.2129117	total:

...

1240:	learn: 0.1846996	total: 3m 3s	remaining: 1m 52s
1241:	learn: 0.1846685	total: 3m 3s	remaining: 1m 51s
1242:	learn: 0.1846320	total: 3m 3s	remaining: 1m 51s
1243:	learn: 0.1845908	total: 3m 3s	remaining: 1m 51s
1244:	learn: 0.1845626	total: 3m 3s	remaining: 1m 51s
1245:	learn: 0.1845194	total: 3m 4s	remaining: 1m 51s
1246:	learn: 0.1844804	total: 3m 4s	remaining: 1m 51s
1247:	learn: 0.1844404	total: 3m 4s	remaining: 1m 51s
1248:	learn: 0.1844049	total: 3m 4s	remaining: 1m 50s
1249:	learn: 0.1843747	total: 3m 4s	remaining: 1m 50s
1250:	learn: 0.1843389	total: 3m 4s	remaining: 1m 50s
1251:	learn: 0.1843001	total: 3m 4s	remaining: 1m 50s
1252:	learn: 0.1842653	total: 3m 5s	remaining: 1m 50s
1253:	learn: 0.1842298	total: 3m 5s	remaining: 1m 50s
1254:	learn: 0.1841949	total: 3m 5s	remaining: 1m 50s
1255:	learn: 0.1841529	total: 3m 5s	remaining: 1m 49s
1256:	learn: 0.1841164	total: 3m 5s	remaining: 1m 49s
1257:	learn: 0.1840828	total: 3m 5s	remaining: 1m 49s
1258:	learn: 0.1840550	total

1390:	learn: 0.1796737	total: 3m 25s	remaining: 1m 29s
1391:	learn: 0.1796419	total: 3m 25s	remaining: 1m 29s
1392:	learn: 0.1796114	total: 3m 25s	remaining: 1m 29s
1393:	learn: 0.1795791	total: 3m 25s	remaining: 1m 29s
1394:	learn: 0.1795442	total: 3m 25s	remaining: 1m 29s
1395:	learn: 0.1795095	total: 3m 26s	remaining: 1m 29s
1396:	learn: 0.1794898	total: 3m 26s	remaining: 1m 29s
1397:	learn: 0.1794600	total: 3m 26s	remaining: 1m 28s
1398:	learn: 0.1794233	total: 3m 26s	remaining: 1m 28s
1399:	learn: 0.1793924	total: 3m 26s	remaining: 1m 28s
1400:	learn: 0.1793569	total: 3m 26s	remaining: 1m 28s
1401:	learn: 0.1793312	total: 3m 27s	remaining: 1m 28s
1402:	learn: 0.1792981	total: 3m 27s	remaining: 1m 28s
1403:	learn: 0.1792575	total: 3m 27s	remaining: 1m 28s
1404:	learn: 0.1792315	total: 3m 27s	remaining: 1m 27s
1405:	learn: 0.1792020	total: 3m 27s	remaining: 1m 27s
1406:	learn: 0.1791726	total: 3m 27s	remaining: 1m 27s
1407:	learn: 0.1791345	total: 3m 27s	remaining: 1m 27s
1408:	lear

1540:	learn: 0.1750944	total: 3m 47s	remaining: 1m 7s
1541:	learn: 0.1750632	total: 3m 47s	remaining: 1m 7s
1542:	learn: 0.1750410	total: 3m 47s	remaining: 1m 7s
1543:	learn: 0.1750110	total: 3m 47s	remaining: 1m 7s
1544:	learn: 0.1749719	total: 3m 47s	remaining: 1m 7s
1545:	learn: 0.1749446	total: 3m 48s	remaining: 1m 6s
1546:	learn: 0.1749152	total: 3m 48s	remaining: 1m 6s
1547:	learn: 0.1748877	total: 3m 48s	remaining: 1m 6s
1548:	learn: 0.1748578	total: 3m 48s	remaining: 1m 6s
1549:	learn: 0.1748216	total: 3m 48s	remaining: 1m 6s
1550:	learn: 0.1747932	total: 3m 48s	remaining: 1m 6s
1551:	learn: 0.1747673	total: 3m 48s	remaining: 1m 6s
1552:	learn: 0.1747377	total: 3m 49s	remaining: 1m 5s
1553:	learn: 0.1747023	total: 3m 49s	remaining: 1m 5s
1554:	learn: 0.1746771	total: 3m 49s	remaining: 1m 5s
1555:	learn: 0.1746438	total: 3m 49s	remaining: 1m 5s
1556:	learn: 0.1746111	total: 3m 49s	remaining: 1m 5s
1557:	learn: 0.1745834	total: 3m 49s	remaining: 1m 5s
1558:	learn: 0.1745567	total

1696:	learn: 0.1706388	total: 4m 10s	remaining: 44.7s
1697:	learn: 0.1706099	total: 4m 10s	remaining: 44.5s
1698:	learn: 0.1705895	total: 4m 10s	remaining: 44.4s
1699:	learn: 0.1705610	total: 4m 10s	remaining: 44.2s
1700:	learn: 0.1705281	total: 4m 10s	remaining: 44.1s
1701:	learn: 0.1704962	total: 4m 10s	remaining: 43.9s
1702:	learn: 0.1704746	total: 4m 11s	remaining: 43.8s
1703:	learn: 0.1704499	total: 4m 11s	remaining: 43.6s
1704:	learn: 0.1704231	total: 4m 11s	remaining: 43.5s
1705:	learn: 0.1703919	total: 4m 11s	remaining: 43.3s
1706:	learn: 0.1703600	total: 4m 11s	remaining: 43.2s
1707:	learn: 0.1703359	total: 4m 11s	remaining: 43.1s
1708:	learn: 0.1703024	total: 4m 11s	remaining: 42.9s
1709:	learn: 0.1702727	total: 4m 12s	remaining: 42.8s
1710:	learn: 0.1702464	total: 4m 12s	remaining: 42.6s
1711:	learn: 0.1702213	total: 4m 12s	remaining: 42.5s
1712:	learn: 0.1701957	total: 4m 12s	remaining: 42.3s
1713:	learn: 0.1701726	total: 4m 12s	remaining: 42.2s
1714:	learn: 0.1701527	total

1850:	learn: 0.1665499	total: 4m 32s	remaining: 21.9s
1851:	learn: 0.1665300	total: 4m 32s	remaining: 21.8s
1852:	learn: 0.1665040	total: 4m 32s	remaining: 21.7s
1853:	learn: 0.1664805	total: 4m 33s	remaining: 21.5s
1854:	learn: 0.1664625	total: 4m 33s	remaining: 21.4s
1855:	learn: 0.1664343	total: 4m 33s	remaining: 21.2s
1856:	learn: 0.1664182	total: 4m 33s	remaining: 21.1s
1857:	learn: 0.1663949	total: 4m 33s	remaining: 20.9s
1858:	learn: 0.1663750	total: 4m 33s	remaining: 20.8s
1859:	learn: 0.1663489	total: 4m 33s	remaining: 20.6s
1860:	learn: 0.1663226	total: 4m 34s	remaining: 20.5s
1861:	learn: 0.1662890	total: 4m 34s	remaining: 20.3s
1862:	learn: 0.1662689	total: 4m 34s	remaining: 20.2s
1863:	learn: 0.1662421	total: 4m 34s	remaining: 20s
1864:	learn: 0.1662213	total: 4m 34s	remaining: 19.9s
1865:	learn: 0.1661956	total: 4m 34s	remaining: 19.7s
1866:	learn: 0.1661634	total: 4m 34s	remaining: 19.6s
1867:	learn: 0.1661371	total: 4m 35s	remaining: 19.4s
1868:	learn: 0.1661136	total: 

In [83]:
y_pred_std = CAT_std.predict_proba(x_test_std) # for classifier
print(y_pred_std[:5])

[[0.98861992 0.01138008]
 [0.83970891 0.16029109]
 [0.98168692 0.01831308]
 [0.9043182  0.0956818 ]
 [0.61015241 0.38984759]]


In [75]:
roc_test = roc_auc_score(y_test_std, y_pred_std[:, 1:])
print(roc_test)

0.8959326887946091


In [84]:
prediction_std = CAT_std.predict_proba(df_test_std)
print(len(prediction_std))

prediction_CAT_std = [to_binary(x[0]) for x in prediction_std[:, 1:]]
print(sum(prediction_CAT_std))
submission = pd.read_csv('./input/sample_submission.csv')
submission['target'] = prediction_CAT_std
submission.to_csv('CAT_std.csv', index=False)

200000
5094


#### MinMax Scaler

In [88]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
df_train_mm = scaler.fit_transform(df_train.iloc[:,2:])
df_test_mm = scaler.transform(df_test.iloc[:,1:])

In [89]:
x_train_mm, x_test_mm, y_train_mm, y_test_mm = train_test_split(df_train_mm, df_train_target, random_state = 13, test_size = 0.3)


In [90]:
CAT_mm = catboost.CatBoostClassifier(iterations=2000, learning_rate=0.03, depth=7)

In [91]:
CAT_mm.fit(x_train_mm, y_train_mm, verbose=True)

0:	learn: 0.6667398	total: 175ms	remaining: 5m 49s
1:	learn: 0.6420957	total: 332ms	remaining: 5m 31s
2:	learn: 0.6194617	total: 506ms	remaining: 5m 36s
3:	learn: 0.5984346	total: 700ms	remaining: 5m 49s
4:	learn: 0.5786616	total: 866ms	remaining: 5m 45s
5:	learn: 0.5602028	total: 1.03s	remaining: 5m 44s
6:	learn: 0.5435402	total: 1.2s	remaining: 5m 42s
7:	learn: 0.5279247	total: 1.38s	remaining: 5m 44s
8:	learn: 0.5132721	total: 1.58s	remaining: 5m 50s
9:	learn: 0.4997661	total: 1.74s	remaining: 5m 45s
10:	learn: 0.4870565	total: 1.91s	remaining: 5m 44s
11:	learn: 0.4747521	total: 2.07s	remaining: 5m 42s
12:	learn: 0.4635358	total: 2.25s	remaining: 5m 44s
13:	learn: 0.4529396	total: 2.42s	remaining: 5m 42s
14:	learn: 0.4435218	total: 2.6s	remaining: 5m 43s
15:	learn: 0.4348249	total: 2.76s	remaining: 5m 42s
16:	learn: 0.4261556	total: 2.9s	remaining: 5m 37s
17:	learn: 0.4180833	total: 3.06s	remaining: 5m 36s
18:	learn: 0.4104215	total: 3.21s	remaining: 5m 34s
19:	learn: 0.4035053	tota

158:	learn: 0.2684802	total: 23.3s	remaining: 4m 29s
159:	learn: 0.2682568	total: 23.4s	remaining: 4m 29s
160:	learn: 0.2680368	total: 23.6s	remaining: 4m 29s
161:	learn: 0.2678017	total: 23.7s	remaining: 4m 29s
162:	learn: 0.2675668	total: 23.9s	remaining: 4m 29s
163:	learn: 0.2673275	total: 24s	remaining: 4m 29s
164:	learn: 0.2671038	total: 24.2s	remaining: 4m 28s
165:	learn: 0.2668784	total: 24.3s	remaining: 4m 28s
166:	learn: 0.2666527	total: 24.4s	remaining: 4m 28s
167:	learn: 0.2664336	total: 24.6s	remaining: 4m 28s
168:	learn: 0.2662304	total: 24.7s	remaining: 4m 27s
169:	learn: 0.2660273	total: 24.9s	remaining: 4m 27s
170:	learn: 0.2658144	total: 25s	remaining: 4m 27s
171:	learn: 0.2656020	total: 25.2s	remaining: 4m 27s
172:	learn: 0.2653865	total: 25.3s	remaining: 4m 27s
173:	learn: 0.2651769	total: 25.4s	remaining: 4m 26s
174:	learn: 0.2649791	total: 25.6s	remaining: 4m 26s
175:	learn: 0.2647753	total: 25.7s	remaining: 4m 26s
176:	learn: 0.2645627	total: 25.9s	remaining: 4m 2

315:	learn: 0.2427071	total: 48.9s	remaining: 4m 20s
316:	learn: 0.2425773	total: 49s	remaining: 4m 20s
317:	learn: 0.2424612	total: 49.2s	remaining: 4m 20s
318:	learn: 0.2423395	total: 49.4s	remaining: 4m 20s
319:	learn: 0.2422252	total: 49.5s	remaining: 4m 20s
320:	learn: 0.2420981	total: 49.7s	remaining: 4m 19s
321:	learn: 0.2419728	total: 49.8s	remaining: 4m 19s
322:	learn: 0.2418468	total: 50s	remaining: 4m 19s
323:	learn: 0.2417111	total: 50.2s	remaining: 4m 19s
324:	learn: 0.2415889	total: 50.4s	remaining: 4m 19s
325:	learn: 0.2414647	total: 50.5s	remaining: 4m 19s
326:	learn: 0.2413503	total: 50.7s	remaining: 4m 19s
327:	learn: 0.2412196	total: 50.9s	remaining: 4m 19s
328:	learn: 0.2410959	total: 51.1s	remaining: 4m 19s
329:	learn: 0.2409819	total: 51.2s	remaining: 4m 19s
330:	learn: 0.2408566	total: 51.4s	remaining: 4m 19s
331:	learn: 0.2407482	total: 51.6s	remaining: 4m 19s
332:	learn: 0.2406368	total: 51.8s	remaining: 4m 19s
333:	learn: 0.2405196	total: 51.9s	remaining: 4m 1

471:	learn: 0.2263873	total: 1m 15s	remaining: 4m 2s
472:	learn: 0.2262970	total: 1m 15s	remaining: 4m 2s
473:	learn: 0.2262107	total: 1m 15s	remaining: 4m 2s
474:	learn: 0.2261289	total: 1m 15s	remaining: 4m 2s
475:	learn: 0.2260386	total: 1m 15s	remaining: 4m 2s
476:	learn: 0.2259562	total: 1m 15s	remaining: 4m 2s
477:	learn: 0.2258677	total: 1m 16s	remaining: 4m 2s
478:	learn: 0.2257770	total: 1m 16s	remaining: 4m 1s
479:	learn: 0.2256939	total: 1m 16s	remaining: 4m 1s
480:	learn: 0.2255925	total: 1m 16s	remaining: 4m 1s
481:	learn: 0.2255076	total: 1m 16s	remaining: 4m 1s
482:	learn: 0.2254247	total: 1m 16s	remaining: 4m 1s
483:	learn: 0.2253428	total: 1m 17s	remaining: 4m 1s
484:	learn: 0.2252650	total: 1m 17s	remaining: 4m 1s
485:	learn: 0.2251716	total: 1m 17s	remaining: 4m
486:	learn: 0.2250812	total: 1m 17s	remaining: 4m
487:	learn: 0.2249973	total: 1m 17s	remaining: 4m
488:	learn: 0.2249210	total: 1m 17s	remaining: 4m
489:	learn: 0.2248271	total: 1m 17s	remaining: 4m
490:	lea

624:	learn: 0.2144555	total: 1m 41s	remaining: 3m 43s
625:	learn: 0.2143795	total: 1m 41s	remaining: 3m 43s
626:	learn: 0.2143023	total: 1m 41s	remaining: 3m 43s
627:	learn: 0.2142239	total: 1m 42s	remaining: 3m 43s
628:	learn: 0.2141539	total: 1m 42s	remaining: 3m 42s
629:	learn: 0.2140876	total: 1m 42s	remaining: 3m 42s
630:	learn: 0.2140239	total: 1m 42s	remaining: 3m 42s
631:	learn: 0.2139541	total: 1m 42s	remaining: 3m 42s
632:	learn: 0.2138888	total: 1m 42s	remaining: 3m 42s
633:	learn: 0.2138113	total: 1m 43s	remaining: 3m 42s
634:	learn: 0.2137371	total: 1m 43s	remaining: 3m 42s
635:	learn: 0.2136617	total: 1m 43s	remaining: 3m 41s
636:	learn: 0.2135913	total: 1m 43s	remaining: 3m 41s
637:	learn: 0.2135171	total: 1m 43s	remaining: 3m 41s
638:	learn: 0.2134441	total: 1m 44s	remaining: 3m 41s
639:	learn: 0.2133840	total: 1m 44s	remaining: 3m 41s
640:	learn: 0.2133183	total: 1m 44s	remaining: 3m 41s
641:	learn: 0.2132483	total: 1m 44s	remaining: 3m 41s
642:	learn: 0.2131737	total:

...

1385:	learn: 0.1798300	total: 3m 47s	remaining: 1m 40s
1386:	learn: 0.1798078	total: 3m 48s	remaining: 1m 40s
1387:	learn: 0.1797752	total: 3m 48s	remaining: 1m 40s
1388:	learn: 0.1797383	total: 3m 48s	remaining: 1m 40s
1389:	learn: 0.1797014	total: 3m 48s	remaining: 1m 40s
1390:	learn: 0.1796737	total: 3m 48s	remaining: 1m 40s
1391:	learn: 0.1796419	total: 3m 48s	remaining: 1m 39s
1392:	learn: 0.1796114	total: 3m 48s	remaining: 1m 39s
1393:	learn: 0.1795791	total: 3m 49s	remaining: 1m 39s
1394:	learn: 0.1795442	total: 3m 49s	remaining: 1m 39s
1395:	learn: 0.1795095	total: 3m 49s	remaining: 1m 39s
1396:	learn: 0.1794898	total: 3m 49s	remaining: 1m 39s
1397:	learn: 0.1794600	total: 3m 49s	remaining: 1m 38s
1398:	learn: 0.1794233	total: 3m 49s	remaining: 1m 38s
1399:	learn: 0.1793924	total: 3m 50s	remaining: 1m 38s
1400:	learn: 0.1793569	total: 3m 50s	remaining: 1m 38s
1401:	learn: 0.1793312	total: 3m 50s	remaining: 1m 38s
1402:	learn: 0.1792981	total: 3m 50s	remaining: 1m 38s
1403:	lear

1536:	learn: 0.1752170	total: 4m 11s	remaining: 1m 15s
1537:	learn: 0.1751824	total: 4m 11s	remaining: 1m 15s
1538:	learn: 0.1751531	total: 4m 11s	remaining: 1m 15s
1539:	learn: 0.1751258	total: 4m 11s	remaining: 1m 15s
1540:	learn: 0.1750944	total: 4m 11s	remaining: 1m 15s
1541:	learn: 0.1750632	total: 4m 12s	remaining: 1m 14s
1542:	learn: 0.1750410	total: 4m 12s	remaining: 1m 14s
1543:	learn: 0.1750110	total: 4m 12s	remaining: 1m 14s
1544:	learn: 0.1749719	total: 4m 12s	remaining: 1m 14s
1545:	learn: 0.1749446	total: 4m 12s	remaining: 1m 14s
1546:	learn: 0.1749152	total: 4m 12s	remaining: 1m 14s
1547:	learn: 0.1748877	total: 4m 12s	remaining: 1m 13s
1548:	learn: 0.1748578	total: 4m 13s	remaining: 1m 13s
1549:	learn: 0.1748216	total: 4m 13s	remaining: 1m 13s
1550:	learn: 0.1747932	total: 4m 13s	remaining: 1m 13s
1551:	learn: 0.1747673	total: 4m 13s	remaining: 1m 13s
1552:	learn: 0.1747377	total: 4m 13s	remaining: 1m 13s
1553:	learn: 0.1747023	total: 4m 13s	remaining: 1m 12s
1554:	lear

1689:	learn: 0.1708400	total: 4m 36s	remaining: 50.7s
1690:	learn: 0.1708110	total: 4m 36s	remaining: 50.5s
1691:	learn: 0.1707748	total: 4m 36s	remaining: 50.4s
1692:	learn: 0.1707494	total: 4m 36s	remaining: 50.2s
1693:	learn: 0.1707152	total: 4m 37s	remaining: 50.1s
1694:	learn: 0.1706897	total: 4m 37s	remaining: 49.9s
1695:	learn: 0.1706716	total: 4m 37s	remaining: 49.7s
1696:	learn: 0.1706388	total: 4m 37s	remaining: 49.6s
1697:	learn: 0.1706099	total: 4m 37s	remaining: 49.4s
1698:	learn: 0.1705895	total: 4m 37s	remaining: 49.2s
1699:	learn: 0.1705610	total: 4m 38s	remaining: 49.1s
1700:	learn: 0.1705281	total: 4m 38s	remaining: 48.9s
1701:	learn: 0.1704962	total: 4m 38s	remaining: 48.8s
1702:	learn: 0.1704746	total: 4m 38s	remaining: 48.6s
1703:	learn: 0.1704499	total: 4m 38s	remaining: 48.4s
1704:	learn: 0.1704231	total: 4m 38s	remaining: 48.3s
1705:	learn: 0.1703919	total: 4m 39s	remaining: 48.1s
1706:	learn: 0.1703600	total: 4m 39s	remaining: 47.9s
1707:	learn: 0.1703359	total

1842:	learn: 0.1667551	total: 5m 1s	remaining: 25.7s
1843:	learn: 0.1667261	total: 5m 1s	remaining: 25.5s
1844:	learn: 0.1667007	total: 5m 2s	remaining: 25.4s
1845:	learn: 0.1666744	total: 5m 2s	remaining: 25.2s
1846:	learn: 0.1666500	total: 5m 2s	remaining: 25s
1847:	learn: 0.1666237	total: 5m 2s	remaining: 24.9s
1848:	learn: 0.1665991	total: 5m 2s	remaining: 24.7s
1849:	learn: 0.1665744	total: 5m 2s	remaining: 24.6s
1850:	learn: 0.1665499	total: 5m 3s	remaining: 24.4s
1851:	learn: 0.1665300	total: 5m 3s	remaining: 24.2s
1852:	learn: 0.1665040	total: 5m 3s	remaining: 24.1s
1853:	learn: 0.1664805	total: 5m 3s	remaining: 23.9s
1854:	learn: 0.1664625	total: 5m 3s	remaining: 23.7s
1855:	learn: 0.1664343	total: 5m 3s	remaining: 23.6s
1856:	learn: 0.1664182	total: 5m 4s	remaining: 23.4s
1857:	learn: 0.1663949	total: 5m 4s	remaining: 23.2s
1858:	learn: 0.1663750	total: 5m 4s	remaining: 23.1s
1859:	learn: 0.1663489	total: 5m 4s	remaining: 22.9s
1860:	learn: 0.1663226	total: 5m 4s	remaining: 2

1996:	learn: 0.1629534	total: 5m 27s	remaining: 491ms
1997:	learn: 0.1629260	total: 5m 27s	remaining: 328ms
1998:	learn: 0.1629094	total: 5m 27s	remaining: 164ms
1999:	learn: 0.1628779	total: 5m 27s	remaining: 0us


In [92]:
y_pred_mm = CAT_mm.predict_proba(x_test_mm) # for classifier
print(y_pred_mm[:5])

[[0.98861992 0.01138008]
 [0.83970891 0.16029109]
 [0.98168692 0.01831308]
 [0.9043182  0.0956818 ]
 [0.61015241 0.38984759]]


In [93]:
roc_test = roc_auc_score(y_test_mm, y_pred_mm[:, 1:])
print(roc_test)

0.8959333276859396


In [94]:
prediction_mm = CAT_mm.predict_proba(df_test_mm)
print(len(prediction_mm))

prediction_CAT_mm = [to_binary(x[0]) for x in prediction_mm[:, 1:]]
print(sum(prediction_CAT_mm))

200000
5095


#### CAT Cross validation

In [95]:
CAT_cross = catboost.CatBoostClassifier(iterations=2000, learning_rate=0.03, depth=7)

In [97]:
from sklearn.model_selection import cross_val_predict

y_pred_cross = cross_val_predict(CAT_cross, df_train.iloc[:,2:], df_train_target, cv=7, verbose=1000)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
0:	learn: 0.6671193	total: 213ms	remaining: 7m 5s
1:	learn: 0.6426426	total: 428ms	remaining: 7m 7s
2:	learn: 0.6201980	total: 612ms	remaining: 6m 47s
3:	learn: 0.5990711	total: 824ms	remaining: 6m 51s
4:	learn: 0.5795738	total: 1.03s	remaining: 6m 53s
5:	learn: 0.5612891	total: 1.24s	remaining: 6m 50s
6:	learn: 0.5439287	total: 1.42s	remaining: 6m 45s
7:	learn: 0.5278251	total: 1.64s	remaining: 6m 47s
8:	learn: 0.5129012	total: 1.84s	remaining: 6m 46s
9:	learn: 0.4988455	total: 2.02s	remaining: 6m 41s
10:	learn: 0.4860041	total: 2.22s	remaining: 6m 41s
11:	learn: 0.4741275	total: 2.4s	remaining: 6m 38s
12:	learn: 0.4629263	total: 2.62s	remaining: 6m 39s
13:	learn: 0.4527055	total: 2.78s	remaining: 6m 35s
14:	learn: 0.4430359	total: 2.97s	remaining: 6m 33s
15:	learn: 0.4337977	total: 3.14s	remaining: 6m 29s
16:	learn: 0.4255375	total: 3.32s	remaining: 6m 27s
17:	learn: 0.4174955	total: 3.48s	remaining: 6m 

157:	learn: 0.2695326	total: 25.6s	remaining: 4m 58s
158:	learn: 0.2693021	total: 25.8s	remaining: 4m 58s
159:	learn: 0.2690767	total: 26s	remaining: 4m 58s
160:	learn: 0.2688613	total: 26.1s	remaining: 4m 58s
161:	learn: 0.2686596	total: 26.3s	remaining: 4m 58s
162:	learn: 0.2684507	total: 26.5s	remaining: 4m 58s
163:	learn: 0.2682362	total: 26.6s	remaining: 4m 58s
164:	learn: 0.2680062	total: 26.8s	remaining: 4m 58s
165:	learn: 0.2678221	total: 27s	remaining: 4m 58s
166:	learn: 0.2676018	total: 27.2s	remaining: 4m 58s
167:	learn: 0.2673878	total: 27.3s	remaining: 4m 57s
168:	learn: 0.2671715	total: 27.5s	remaining: 4m 57s
169:	learn: 0.2669553	total: 27.6s	remaining: 4m 57s
170:	learn: 0.2667468	total: 27.8s	remaining: 4m 57s
171:	learn: 0.2665423	total: 28s	remaining: 4m 57s
172:	learn: 0.2663367	total: 28.1s	remaining: 4m 57s
173:	learn: 0.2661211	total: 28.3s	remaining: 4m 56s
174:	learn: 0.2659210	total: 28.4s	remaining: 4m 56s
175:	learn: 0.2657135	total: 28.6s	remaining: 4m 56s

313:	learn: 0.2440999	total: 51.3s	remaining: 4m 35s
314:	learn: 0.2439776	total: 51.4s	remaining: 4m 35s
315:	learn: 0.2438452	total: 51.6s	remaining: 4m 34s
316:	learn: 0.2437285	total: 51.7s	remaining: 4m 34s
317:	learn: 0.2436055	total: 51.9s	remaining: 4m 34s
318:	learn: 0.2434874	total: 52.1s	remaining: 4m 34s
319:	learn: 0.2433686	total: 52.3s	remaining: 4m 34s
320:	learn: 0.2432493	total: 52.4s	remaining: 4m 34s
321:	learn: 0.2431395	total: 52.6s	remaining: 4m 33s
322:	learn: 0.2430120	total: 52.7s	remaining: 4m 33s
323:	learn: 0.2428920	total: 52.9s	remaining: 4m 33s
324:	learn: 0.2427880	total: 53.1s	remaining: 4m 33s
325:	learn: 0.2426608	total: 53.2s	remaining: 4m 33s
326:	learn: 0.2425304	total: 53.4s	remaining: 4m 33s
327:	learn: 0.2424168	total: 53.6s	remaining: 4m 33s
328:	learn: 0.2423063	total: 53.7s	remaining: 4m 32s
329:	learn: 0.2421900	total: 53.9s	remaining: 4m 32s
330:	learn: 0.2420800	total: 54s	remaining: 4m 32s
331:	learn: 0.2419628	total: 54.2s	remaining: 4m

469:	learn: 0.2282531	total: 1m 16s	remaining: 4m 10s
470:	learn: 0.2281663	total: 1m 17s	remaining: 4m 10s
471:	learn: 0.2280800	total: 1m 17s	remaining: 4m 10s
472:	learn: 0.2279979	total: 1m 17s	remaining: 4m 10s
473:	learn: 0.2279170	total: 1m 17s	remaining: 4m 9s
474:	learn: 0.2278250	total: 1m 17s	remaining: 4m 9s
475:	learn: 0.2277325	total: 1m 17s	remaining: 4m 9s
476:	learn: 0.2276495	total: 1m 18s	remaining: 4m 9s
477:	learn: 0.2275696	total: 1m 18s	remaining: 4m 9s
478:	learn: 0.2274859	total: 1m 18s	remaining: 4m 9s
479:	learn: 0.2274079	total: 1m 18s	remaining: 4m 9s
480:	learn: 0.2273212	total: 1m 18s	remaining: 4m 8s
481:	learn: 0.2272394	total: 1m 18s	remaining: 4m 8s
482:	learn: 0.2271512	total: 1m 19s	remaining: 4m 8s
483:	learn: 0.2270699	total: 1m 19s	remaining: 4m 8s
484:	learn: 0.2269844	total: 1m 19s	remaining: 4m 8s
485:	learn: 0.2269040	total: 1m 19s	remaining: 4m 8s
486:	learn: 0.2268273	total: 1m 19s	remaining: 4m 7s
487:	learn: 0.2267434	total: 1m 19s	remain

623:	learn: 0.2168713	total: 1m 42s	remaining: 3m 46s
624:	learn: 0.2168009	total: 1m 42s	remaining: 3m 45s
625:	learn: 0.2167373	total: 1m 42s	remaining: 3m 45s
626:	learn: 0.2166776	total: 1m 43s	remaining: 3m 45s
627:	learn: 0.2166104	total: 1m 43s	remaining: 3m 45s
628:	learn: 0.2165383	total: 1m 43s	remaining: 3m 45s
629:	learn: 0.2164708	total: 1m 43s	remaining: 3m 45s
630:	learn: 0.2164077	total: 1m 43s	remaining: 3m 45s
631:	learn: 0.2163358	total: 1m 43s	remaining: 3m 44s
632:	learn: 0.2162685	total: 1m 44s	remaining: 3m 44s
633:	learn: 0.2162100	total: 1m 44s	remaining: 3m 44s
634:	learn: 0.2161477	total: 1m 44s	remaining: 3m 44s
635:	learn: 0.2160871	total: 1m 44s	remaining: 3m 44s
636:	learn: 0.2160249	total: 1m 44s	remaining: 3m 44s
637:	learn: 0.2159674	total: 1m 44s	remaining: 3m 43s
638:	learn: 0.2158952	total: 1m 45s	remaining: 3m 43s
639:	learn: 0.2158358	total: 1m 45s	remaining: 3m 43s
640:	learn: 0.2157812	total: 1m 45s	remaining: 3m 43s
641:	learn: 0.2157158	total:

...

1369:	learn: 0.1856277	total: 3m 54s	remaining: 1m 47s
1370:	learn: 0.1856038	total: 3m 54s	remaining: 1m 47s
1371:	learn: 0.1855721	total: 3m 54s	remaining: 1m 47s
1372:	learn: 0.1855418	total: 3m 55s	remaining: 1m 47s
1373:	learn: 0.1855161	total: 3m 55s	remaining: 1m 47s
1374:	learn: 0.1854799	total: 3m 55s	remaining: 1m 47s
1375:	learn: 0.1854523	total: 3m 55s	remaining: 1m 46s
1376:	learn: 0.1854185	total: 3m 55s	remaining: 1m 46s
1377:	learn: 0.1853890	total: 3m 55s	remaining: 1m 46s
1378:	learn: 0.1853552	total: 3m 56s	remaining: 1m 46s
1379:	learn: 0.1853255	total: 3m 56s	remaining: 1m 46s
1380:	learn: 0.1852995	total: 3m 56s	remaining: 1m 46s
1381:	learn: 0.1852750	total: 3m 56s	remaining: 1m 45s
1382:	learn: 0.1852490	total: 3m 56s	remaining: 1m 45s
1383:	learn: 0.1852243	total: 3m 57s	remaining: 1m 45s
1384:	learn: 0.1851969	total: 3m 57s	remaining: 1m 45s
1385:	learn: 0.1851644	total: 3m 57s	remaining: 1m 45s
1386:	learn: 0.1851385	total: 3m 57s	remaining: 1m 45s
1387:	lear

1520:	learn: 0.1812675	total: 4m 21s	remaining: 1m 22s
1521:	learn: 0.1812451	total: 4m 21s	remaining: 1m 22s
1522:	learn: 0.1812239	total: 4m 22s	remaining: 1m 22s
1523:	learn: 0.1811919	total: 4m 22s	remaining: 1m 21s
1524:	learn: 0.1811666	total: 4m 22s	remaining: 1m 21s
1525:	learn: 0.1811355	total: 4m 22s	remaining: 1m 21s
1526:	learn: 0.1811116	total: 4m 22s	remaining: 1m 21s
1527:	learn: 0.1810837	total: 4m 23s	remaining: 1m 21s
1528:	learn: 0.1810606	total: 4m 23s	remaining: 1m 21s
1529:	learn: 0.1810350	total: 4m 23s	remaining: 1m 20s
1530:	learn: 0.1810130	total: 4m 23s	remaining: 1m 20s
1531:	learn: 0.1809869	total: 4m 23s	remaining: 1m 20s
1532:	learn: 0.1809612	total: 4m 23s	remaining: 1m 20s
1533:	learn: 0.1809370	total: 4m 24s	remaining: 1m 20s
1534:	learn: 0.1809170	total: 4m 24s	remaining: 1m 20s
1535:	learn: 0.1808899	total: 4m 24s	remaining: 1m 19s
1536:	learn: 0.1808600	total: 4m 24s	remaining: 1m 19s
1537:	learn: 0.1808284	total: 4m 24s	remaining: 1m 19s
1538:	lear

1671:	learn: 0.1772563	total: 4m 48s	remaining: 56.7s
1672:	learn: 0.1772287	total: 4m 49s	remaining: 56.5s
1673:	learn: 0.1772106	total: 4m 49s	remaining: 56.3s
1674:	learn: 0.1771846	total: 4m 49s	remaining: 56.1s
1675:	learn: 0.1771622	total: 4m 49s	remaining: 56s
1676:	learn: 0.1771352	total: 4m 49s	remaining: 55.8s
1677:	learn: 0.1771137	total: 4m 49s	remaining: 55.6s
1678:	learn: 0.1770915	total: 4m 50s	remaining: 55.5s
1679:	learn: 0.1770624	total: 4m 50s	remaining: 55.3s
1680:	learn: 0.1770413	total: 4m 50s	remaining: 55.1s
1681:	learn: 0.1770200	total: 4m 50s	remaining: 54.9s
1682:	learn: 0.1769923	total: 4m 50s	remaining: 54.8s
1683:	learn: 0.1769700	total: 4m 50s	remaining: 54.6s
1684:	learn: 0.1769518	total: 4m 51s	remaining: 54.4s
1685:	learn: 0.1769330	total: 4m 51s	remaining: 54.2s
1686:	learn: 0.1769104	total: 4m 51s	remaining: 54.1s
1687:	learn: 0.1768785	total: 4m 51s	remaining: 53.9s
1688:	learn: 0.1768564	total: 4m 51s	remaining: 53.7s
1689:	learn: 0.1768298	total: 

1826:	learn: 0.1735789	total: 5m 15s	remaining: 29.9s
1827:	learn: 0.1735540	total: 5m 16s	remaining: 29.7s
1828:	learn: 0.1735292	total: 5m 16s	remaining: 29.6s
1829:	learn: 0.1735084	total: 5m 16s	remaining: 29.4s
1830:	learn: 0.1734879	total: 5m 16s	remaining: 29.2s
1831:	learn: 0.1734652	total: 5m 16s	remaining: 29.1s
1832:	learn: 0.1734456	total: 5m 17s	remaining: 28.9s
1833:	learn: 0.1734284	total: 5m 17s	remaining: 28.7s
1834:	learn: 0.1734030	total: 5m 17s	remaining: 28.5s
1835:	learn: 0.1733787	total: 5m 17s	remaining: 28.4s
1836:	learn: 0.1733567	total: 5m 17s	remaining: 28.2s
1837:	learn: 0.1733379	total: 5m 17s	remaining: 28s
1838:	learn: 0.1733160	total: 5m 18s	remaining: 27.8s
1839:	learn: 0.1732989	total: 5m 18s	remaining: 27.7s
1840:	learn: 0.1732779	total: 5m 18s	remaining: 27.5s
1841:	learn: 0.1732605	total: 5m 18s	remaining: 27.3s
1842:	learn: 0.1732370	total: 5m 18s	remaining: 27.1s
1843:	learn: 0.1732118	total: 5m 18s	remaining: 27s
1844:	learn: 0.1731964	total: 5m

1979:	learn: 0.1701856	total: 5m 42s	remaining: 3.46s
1980:	learn: 0.1701646	total: 5m 42s	remaining: 3.29s
1981:	learn: 0.1701358	total: 5m 42s	remaining: 3.11s
1982:	learn: 0.1701199	total: 5m 43s	remaining: 2.94s
1983:	learn: 0.1700996	total: 5m 43s	remaining: 2.77s
1984:	learn: 0.1700779	total: 5m 43s	remaining: 2.6s
1985:	learn: 0.1700494	total: 5m 43s	remaining: 2.42s
1986:	learn: 0.1700247	total: 5m 43s	remaining: 2.25s
1987:	learn: 0.1700026	total: 5m 44s	remaining: 2.08s
1988:	learn: 0.1699767	total: 5m 44s	remaining: 1.9s
1989:	learn: 0.1699618	total: 5m 44s	remaining: 1.73s
1990:	learn: 0.1699417	total: 5m 44s	remaining: 1.56s
1991:	learn: 0.1699197	total: 5m 44s	remaining: 1.38s
1992:	learn: 0.1698968	total: 5m 44s	remaining: 1.21s
1993:	learn: 0.1698733	total: 5m 45s	remaining: 1.04s
1994:	learn: 0.1698529	total: 5m 45s	remaining: 865ms
1995:	learn: 0.1698339	total: 5m 45s	remaining: 692ms
1996:	learn: 0.1698125	total: 5m 45s	remaining: 519ms
1997:	learn: 0.1697841	total: 

In [109]:
print(int(sum(y_pred_cross))) # submitted score of 0.499

7632


In [116]:
roc_test = roc_auc_score(df_train_target, y_pred_cross)
print(roc_test)

0.6471942886687837


In [118]:
submission = pd.read_csv('./input/sample_submission.csv')
submission['target'] = [to_binary(x) for x in y_pred_cross]
submission.to_csv('CAT_cross.csv', index=False)